In [13]:
import numpy as np
import sys
sys.path.append("../scratch_2")
from common.layers import MatMul

In [3]:
c = np.array([[1,0,0,0,0,0,0]])
W = np.random.randn(7,3)
h = np.matmul(c,W)
print(h)
print(W)

[[-0.12061177  2.01199021  1.90027099]]
[[-0.12061177  2.01199021  1.90027099]
 [ 0.9285594  -0.15668082  0.48758171]
 [ 0.89719587  0.18087077  0.94169314]
 [ 1.87902431 -0.88668164  0.38030872]
 [-0.73923681  1.52389862  0.40944297]
 [ 1.18061369  2.34010011 -2.04424258]
 [ 1.38755454 -0.1717398  -0.72139993]]


가중치의 행벡터 하나를 뽑아낸 것과 같다

In [14]:
layer = MatMul(W)
h = layer.forward(c)
print(h)

[[-0.12061177  2.01199021  1.90027099]]


In [15]:
c0 = np.array([[1,0,0,0,0,0,0]])
c1 = np.array([[0,0,1,0,0,0,0]])

W_in = np.random.randn(7,3)
W_out = np.random.randn(3,7)

in_layer0 = MatMul(W_in)
in_layer1 = MatMul(W_in)
out_layer = MatMul(W_out)

# 순전파 -> 편향이 없는 상태이므로 완전 연결계층 처리가 MatMul layer의 순전파와 같음
h0 = in_layer0.forward(c0)
h1 = in_layer1.forward(c1)
h = 0.5 *(h0+h1)
s = out_layer.forward(h)

print(s)

[[ 0.60616112 -0.080469    0.79890414 -0.23682013 -0.78288412  0.25583451
  -0.99053421]]


### CBOW 모델이란?

CBOW는 continuous bag of words를 말하는데, **맥락으로부터 타깃을 추측하는 용도의 신경망**이다.
여기서 맥락은 타깃 주변단어들을 말한다. CBOW모델을 가능한 정확하게 추론하도록 훈련시켜서 단어들의 분산표현을 얻어내는것이 목표이다.

cf. 타깃으로 부터 맥락을 추측하는 용도의 신경망은 skip-gram이다

따라서 입력이 맥락(주변단어)가 되는데, 이때 형식은 원핫 벡터이다.
모델을 구성할때는 은닉층의 뉴런수가 입력층의 뉴런수보다 적게 하는 것이 핵심인데, 이렇게 하면 밀집벡터 표현을 얻을 수 있기 때문입니다.(sparse matrix형식을 유지하게 되면 메모리 사용량도 크고 계산이 오래걸린다.)

**CBOW 모델의 추론과정**을 간단히 적어보면
1. 필요한 가중치를 초기화 한다.(입력층->은닉층, 은닉층->출력층)
2. 입력층을 처리하는 MatMul 계층을 맥락수 만큼 만든다. (모두 같은 가중치를 사용)
3. 출력층의 MatMul은 하나만 생성한다. (입력층에서 처리된 값을 평균하여 사용하기 때문)
4. 각 계층을 차례로 통과시켜(forward) 각 단어의 점수를 구한다.
5. (점수에 softmax함수를 적용하면 각 단어가 등장할 확률로 표현이 가능하다.)

**CBOW 모델의 학습과정**을 간단히 적어보면,
Softmax 함수와 Cross Entropy Error 계층을 추가하여 loss function으로 사용한다.
loss fuction의 값이 줄어드는 방향으로 가중치를 조절해 나가면 된다. 
